In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
import pprint
%matplotlib inline

import os
print(os.listdir("../input"))

Using TensorFlow backend.


['Colo - Copy (2).csv', 'Colo-Humidity.csv', 'Colo.csv', 'my_model', 'my_model.h5', 'Preciption_(2020)', 'Preciption_(2020).h5', 'Precption(2).csv', 'Precption(3).csv', 'Precption(4).csv', 'Precption.csv']


In [2]:
df = pd.read_csv('../input/Colo-Humidity.csv')

In [3]:
df

,Date,Humidity
0,1/1/2009,62
1,2/1/2009,59
2,3/1/2009,58
3,4/1/2009,66
4,5/1/2009,68
...,...,...
4012,27/12/2019,77
4013,28/12/2019,78
4014,29/12/2019,76
4015,30/12/2019,78


In [4]:
df.Date=pd.to_datetime(df.Date)
df=df.set_index('Date')

In [5]:
df

,Humidity
Date,
2009-01-01,62
2009-02-01,59
2009-03-01,58
2009-04-01,66
2009-05-01,68
...,...
2019-12-27,77
2019-12-28,78
2019-12-29,76


In [6]:
TestData = df.tail(365)

In [7]:
Df_Total = pd.concat((df[["Humidity"]], TestData[["Humidity"]]), axis=0)

In [8]:
Df_Total.shape

(4382, 1)

In [9]:
inputs = Df_Total[len(Df_Total) - len(TestData) - 60:].values
inputs.shape

(425, 1)

In [10]:
from pandas.tseries.offsets import DateOffset
add_dates = [df.index[-1] + DateOffset(days=x) for x in range(0,366)]
future_dates = pd.DataFrame(index=add_dates[1:],columns=df.columns)

In [11]:
future_dates.tail(365)

,Humidity
2020-01-01,NaN
2020-01-02,NaN
2020-01-03,NaN
2020-01-04,NaN
2020-01-05,NaN
...,...
2020-12-26,NaN
2020-12-27,NaN
2020-12-28,NaN
2020-12-29,NaN


In [12]:
df.shape

(4017, 1)

In [13]:
Training_Set = df

In [14]:
Training_Set = Training_Set.values
sc = MinMaxScaler(feature_range=(0, 1))
Train = sc.fit_transform(Training_Set)

In [15]:
Train.shape

(4017, 1)

In [16]:
X_Train = []
Y_Train = []

# Range should be fromm 60 Values to END
for i in range(60, Train.shape[0]):
   
    # X_Train 0-59
    X_Train.append(Train[i-60:i])
   
    # Y Would be 60 th Value based on past 60 Values
    Y_Train.append(Train[i])

# Convert into Numpy Array
X_Train = np.array(X_Train)
Y_Train = np.array(Y_Train)

print(X_Train.shape)
print(Y_Train.shape)

(3957, 60, 1)
(3957, 1)


In [17]:
X_Train = np.reshape(X_Train, newshape=(X_Train.shape[0], X_Train.shape[1], 1))
X_Train.shape

(3957, 60, 1)

# Model

In [18]:
regressor = Sequential()

# Adding the first LSTM layer and some Dropout regularisation
regressor.add(LSTM(units = 150, return_sequences = True,input_shape = (X_Train.shape[1], 1)))
regressor.add(Dropout(0.2))

# Adding a second LSTM layer and some Dropout regularisation
regressor.add(LSTM(units = 150, return_sequences = True))
regressor.add(Dropout(0.2))

# Adding a third LSTM layer and some Dropout regularisation
regressor.add(LSTM(units = 150, return_sequences = True))
regressor.add(Dropout(0.2))

# Adding a fourth LSTM layer and some Dropout regularisation
regressor.add(LSTM(units = 150))
regressor.add(Dropout(0.2))

# Adding the output layer
regressor.add(Dense(units = 1))

In [19]:
regressor.summary()
# Compiling the RNN
regressor.compile(optimizer = 'adam', loss = 'mean_squared_error')

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 60, 150)           91200     
_________________________________________________________________
dropout_1 (Dropout)          (None, 60, 150)           0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 60, 150)           180600    
_________________________________________________________________
dropout_2 (Dropout)          (None, 60, 150)           0         
_________________________________________________________________
lstm_3 (LSTM)                (None, 60, 150)           180600    
_________________________________________________________________
dropout_3 (Dropout)          (None, 60, 150)           0         
_________________________________________________________________
lstm_4 (LSTM)                (None, 150)              

In [20]:
regressor.fit(X_Train,Y_Train, epochs = 100, batch_size = 6)

Epoch 1/100
3957/3957 [==============================] - 152s 38ms/step - loss: 0.0184
Epoch 2/100
3957/3957 [==============================] - 151s 38ms/step - loss: 0.0126
Epoch 3/100
3957/3957 [==============================] - 152s 38ms/step - loss: 0.0093
Epoch 4/100
3957/3957 [==============================] - 154s 39ms/step - loss: 0.0080
Epoch 5/100
3957/3957 [==============================] - 157s 40ms/step - loss: 0.0076
Epoch 6/100
3957/3957 [==============================] - 152s 39ms/step - loss: 0.0072
Epoch 7/100
3957/3957 [==============================] - 152s 38ms/step - loss: 0.0073
Epoch 8/100
3957/3957 [==============================] - 152s 39ms/step - loss: 0.0070
Epoch 9/100
3957/3957 [==============================] - 153s 39ms/step - loss: 0.0070
Epoch 10/100
3957/3957 [==============================] - 153s 39ms/step - loss: 0.0070
Epoch 11/100
3957/3957 [==============================] - 154s 39ms/step - loss: 0.0068
Epoch 12/100
3957/3957 [=================

3957/3957 [==============================] - 156s 39ms/step - loss: 0.0055
Epoch 95/100
3957/3957 [==============================] - 154s 39ms/step - loss: 0.0055
Epoch 96/100
3957/3957 [==============================] - 155s 39ms/step - loss: 0.0055
Epoch 97/100
3957/3957 [==============================] - 154s 39ms/step - loss: 0.0054
Epoch 98/100
3957/3957 [==============================] - 155s 39ms/step - loss: 0.0055
Epoch 99/100
3957/3957 [==============================] - 155s 39ms/step - loss: 0.0054
Epoch 100/100
3957/3957 [==============================] - 156s 39ms/step - loss: 0.0052


In [21]:
future_dates.shape

(365, 1)

In [22]:
inputs = Df_Total[len(Df_Total) - len(TestData) - 60:].values

# We need to Reshape
inputs = inputs.reshape(-1,1)

# Normalize the Dataset
inputs = sc.transform(inputs)

X_test = []
for i in range(60, 425):
    X_test.append(inputs[i-60:i])
       
# Convert into Numpy Array
X_test = np.array(X_test)

# Reshape before Passing to Network
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))

# Pass to Model
predicted_Precip = regressor.predict(X_test)

# Do inverse Transformation to get Values
predicted_Precip = sc.inverse_transform(predicted_Precip)

In [23]:
True_Precip = 0
Predicted_Precip  = predicted_Precip
dates = TestData.index.to_list()

In [24]:
#df_predict = pd.DataFrame(scaler.inverse_transform(pred_list), index=future_dates[-n_input:].index, columns=['Predictions'])
Machine_Df2 = pd.DataFrame(data={
    "Date":future_dates.index.to_list(),
    "TrueHumidity": True_Precip,
    "PredictedHumidity":[x[0] for x in predicted_Precip]
})

In [25]:
Machine_Df2

,Date,TrueHumidity,PredictedHumidity
0,2020-01-01,0,81.155708
1,2020-01-02,0,77.403786
2,2020-01-03,0,76.592415
3,2020-01-04,0,74.987465
4,2020-01-05,0,74.560303
...,...,...,...
360,2020-12-26,0,81.187187
361,2020-12-27,0,77.717102
362,2020-12-28,0,80.200378
363,2020-12-29,0,78.022911


In [30]:
#plt.figure(figsize=(25,15))
#plt.plot(TestData.index, Predicted_Precip)
#plt.show()

In [27]:
Machine_Df2.to_csv("../input/Humidity.csv")

In [28]:
regressor.save('../input/Humidity_(2020)') 

In [29]:
regressor.save('../input/Humidity_(2020).h5') 